# Cold Start - AB 240422 Analysis

**ab_240422** appeared to have the same performance as the previous **best_meme_from_each_source**. The reasons why these results differ from the previous offline simulation

* The main probable reason is the bias in the training data
* In the previous simulation, I removed users that blocked the bot
* In the current simulation I left only to 40 items for each new user

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import hashlib
import json
from datetime import datetime

import numpy as np
import polars as pl
from sklearn.model_selection import train_test_split

In [4]:
from models import ColdStartRecommender, BestMemeFromEachSource, MostLiked
from estimate import estimate_cs, estimate_cs_prod

In [5]:
coldstart_df = pl.read_parquet('coldstart.pq')
meme_features_daily_df = pl.read_parquet('meme_features_daily.pq')

# v2 is calculated on users with less than 200 responses
meme_features_daily_v2_df = pl.read_parquet('meme_features_daily_v2.pq')

In [6]:
coldstart_df.slice(2, 1)

user_id,dtm,date_dtm,target_memes,target_reactions,target_recommended_by
i64,datetime[μs],datetime[μs],list[i64],list[i64],list[str]
211558,2024-03-18 04:07:42.589074,2024-03-18 00:00:00,"[1195419, 171119, … 790223]","[2, 2, … 2]","[""best_meme_from_each_source"", ""best_meme_from_each_source"", … ""best_meme_from_each_source""]"


In [7]:
validation_df = (
    coldstart_df
    .filter(pl.col('date_dtm') >= datetime(2024, 4, 25))
    .filter(pl.col('date_dtm') < datetime(2024, 5, 5))
    .with_columns(pl.col('target_memes').list.head(30))
    .with_columns(pl.col('target_reactions').list.head(30))
    .with_columns(pl.col('target_recommended_by').list.head(30))
)

In [8]:
estimate_cs_prod('best_meme_from_each_source', validation_df)

Likes - 4064, Like Rate = 0.521 +- 0.030, Like Rate Micro = 0.194 +- 0.018


In [9]:
estimate_cs_prod('random_best_ab_240422', validation_df)

Likes - 3036, Like Rate = 0.520 +- 0.033, Like Rate Micro = 0.151 +- 0.017


In [24]:
best_meme_from_each_source = BestMemeFromEachSource(meme_features_daily_df)
estimate_cs(best_meme_from_each_source, validation_df, top_size=100)

Likes - 284, Like Rate = 0.577 +- 0.052, Like Rate Micro = 0.546 +- 0.053


In [25]:
most_liked = MostLiked(meme_features_daily_df)
estimate_cs(most_liked, validation_df, top_size=100)

Likes - 266, Like Rate = 0.583 +- 0.074, Like Rate Micro = 0.495 +- 0.071


In [26]:
most_liked_v2 = MostLiked(meme_features_daily_v2_df)
estimate_cs(most_liked_v2, validation_df, top_size=100)

Likes - 342, Like Rate = 0.589 +- 0.052, Like Rate Micro = 0.542 +- 0.053
